<a href="https://colab.research.google.com/github/savarirohan3-byte/AIDS-Learning/blob/main/chat_bot_travel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow pandas numpy scikit-learn

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
data = pd.read_csv(r"Top Indian Places to Visit.csv")

data = data.sample(frac=1)   # shuffle
data.head()

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
238,238,Southern,Tamil Nadu,Rameswaram,Ramanathaswamy Temple,Temple,12th century,1.5,4.6,0,No,NaN,Religious,No,0.010,All
29,29,Southern,Karnataka,Bangalore,Cubbon Park,Park,1870,1.0,4.4,0,Yes,NaN,Nature,Yes,1.320,Morning
235,235,Eastern,Odisha,Kendujhar,Sanaghagara Waterfall,Waterfall,Unknown,1.0,4.4,0,No,NaN,Nature,Yes,0.055,All
62,62,Southern,Goa,Goa,Colva Beach,Beach,Unknown,2.0,4.3,0,Yes,NaN,Scenic,Yes,0.100,Evening
281,281,North Eastern,Arunachal Pradesh,Tawang,Tawang Monastery,Monastery,1680,2.0,4.7,0,No,NaN,Religious,No,0.032,Morning


In [4]:
stories = []
questions = []
answers = []

for i in range(len(data)):

    place = str(data.loc[i,"Name"]).lower()
    state = str(data.loc[i,"State"]).lower()
    city = str(data.loc[i,"City"]).lower()
    typ = str(data.loc[i,"Type"]).lower()
    best_time = str(data.loc[i,"Best Time to visit"]).lower()
    rating = str(data.loc[i,"Google review rating"]).lower()
    fee = str(data.loc[i,"Entrance Fee in INR"]).lower()

    story = f"{place} is in {city} {state}. it is a {typ}. best time to visit is {best_time}. rating is {rating}. entrance fee is {fee}."

    # LOCATION
    questions.extend([
        f"where is {place} located",
        f"which state is {place} in",
        f"{place} is in which state"
    ])
    answers.extend([state, state, state])
    stories.extend([story, story, story])

    # TYPE
    questions.append(f"what type of place is {place}")
    answers.append(typ)
    stories.append(story)

    # BEST TIME
    questions.append(f"best time to visit {place}")
    answers.append(best_time)
    stories.append(story)

    # RATING
    questions.append(f"what is rating of {place}")
    answers.append(rating)
    stories.append(story)

    # ENTRY FEE
    questions.append(f"what is entrance fee of {place}")
    answers.append(fee)
    stories.append(story)

In [5]:
all_text = stories + questions + answers

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(all_text)

vocab_size = len(tokenizer.word_index) + 1

In [6]:
story_seq = tokenizer.texts_to_sequences(stories)
question_seq = tokenizer.texts_to_sequences(questions)
answer_seq = tokenizer.texts_to_sequences(answers)

In [7]:
max_story_len = max(len(x) for x in story_seq)
max_question_len = max(len(x) for x in question_seq)

X_story = pad_sequences(story_seq, maxlen=max_story_len)
X_question = pad_sequences(question_seq, maxlen=max_question_len)

In [8]:
answer_tokens = [x[0] for x in answer_seq]

y = to_categorical(answer_tokens, num_classes=vocab_size)

In [9]:
Xs_train, Xs_test, Xq_train, Xq_test, y_train, y_test = train_test_split(
    X_story, X_question, y, test_size=0.2
)

In [10]:
input_story = Input(shape=(max_story_len,))
x1 = Embedding(vocab_size, 128)(input_story)
x1 = Bidirectional(LSTM(64))(x1)

input_question = Input(shape=(max_question_len,))
x2 = Embedding(vocab_size, 128)(input_question)
x2 = Bidirectional(LSTM(64))(x2)

merged = concatenate([x1, x2])

output = Dense(vocab_size, activation="softmax")(merged)

model = Model([input_story, input_question], output)

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 29)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 29, 128)   │     98,944 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 11, 128)   │     98,944 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     98,816 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 128)       │     98,816 │ embedding_1[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 773)       │    198,661 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 594,181 (2.27 MB)

 Trainable params: 594,181 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)

model.fit([Xs_train, Xq_train], y_train,
          batch_size=32,
          epochs=100,
          validation_data=([Xs_test, Xq_test], y_test),
          callbacks=[early_stop])

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.1530 - loss: 5.6735 - val_accuracy: 0.1495 - val_loss: 3.8990
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.2137 - loss: 3.6276 - val_accuracy: 0.3187 - val_loss: 2.9604
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.2960 - loss: 2.7821 - val_accuracy: 0.3758 - val_loss: 2.6709
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.3652 - loss: 2.4158 - val_accuracy: 0.4198 - val_loss: 2.4480
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.4519 - loss: 2.1370 - val_accuracy: 0.5143 - val_loss: 2.1295
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.5480 - loss: 1.7454 - val_accuracy: 0.6044 - val_loss: 1.8468
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.6410 - loss: 1.4413 - val_accuracy: 0.6791 - val_loss: 1.6207
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.7238 - loss: 1.1365 - val_accuracy: 0

In [12]:
model.save("travel_chatbot.keras")

In [13]:
reverse_word_index = {v:k for k,v in tokenizer.word_index.items()}

def get_story(place):

    place_data = data[data["Name"].str.lower() == place.lower()]

    if len(place_data) == 0:
        return None

    row = place_data.iloc[0]

    story = f"{row['Name']} is in {row['City']} {row['State']}. it is a {row['Type']}. best time to visit is {row['Best Time to visit']}. rating is {row['Google review rating']}. entrance fee is {row['Entrance Fee in INR']}."

    return story.lower()


def chatbot():

    while True:

        place = input("\nEnter place name (or exit): ")

        if place.lower() == "exit":
            print("✈️ Thank you for using Travel Bot")
            break

        story = get_story(place)

        if story is None:
            print("❌ Place not found")
            continue

        question = input("Ask your question: ").lower()

        story_seq = tokenizer.texts_to_sequences([story])
        question_seq = tokenizer.texts_to_sequences([question])

        story_pad = pad_sequences(story_seq, maxlen=max_story_len)
        question_pad = pad_sequences(question_seq, maxlen=max_question_len)

        pred = model.predict([story_pad, question_pad])

        idx = np.argmax(pred)

        print("🤖 Answer:", reverse_word_index.get(idx, "Not found"))

In [14]:
chatbot()


Enter place name (or exit): Lotus Temple
Ask your question: Is it closed on any specific day?
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
🤖 Answer: delhi

Enter place name (or exit): Lotus Temple
Ask your question: where is located
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
🤖 Answer: delhi

Enter place name (or exit): akshardham temple
Ask your question: best time to visit akshardham temple
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
🤖 Answer: afternoon

Enter place name (or exit): lotus temple
Ask your question: what is rating of lotus temple
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
🤖 Answer: 4

Enter place name (or exit): exit
✈️ Thank you for using Travel Bot
